In [4]:
#@title -- Installation of Packages -- { display-mode: "form" }
import sys
!{sys.executable} -m pip install gym[classic_control]
!{sys.executable} -m pip install tianshou
!{sys.executable} -m pip install git+https://github.com/michalgregor/tianshou_agents.git
!{sys.executable} -m pip install pygame

     |████████████████████████████████| 144 kB 5.0 MB/s 
  Cloning https://github.com/michalgregor/tianshou_agents.git to /tmp/pip-req-build-2s6pql3_
  Running command git clone -q https://github.com/michalgregor/tianshou_agents.git /tmp/pip-req-build-2s6pql3_
  Created wheel for tianshou-agents: filename=tianshou_agents-0.1-py3-none-any.whl size=49720 sha256=cd17c7bd38e37daaf6425cde019f3f03dc5e3315b4d85ded3a4e794aa836935d
  Stored in directory: /tmp/pip-ephem-wheel-cache-mprq83ik/wheels/cb/ef/e9/8794fa082fcfe8ee6255595117bac656566256f9755186adf0
Successfully built tianshou-agents


In [1]:
#@title -- Import of Necessary Packages -- { display-mode: "form" }
from tianshou_agents.methods.sac import sac_simple
from tianshou_agents.components import PassiveCollector
from tianshou.data import Batch
from tianshou.env import DummyVectorEnv
import numpy as np
import gym

/usr/local/lib/python3.7/dist-packages/numba/core/types/__init__.py:108: DeprecationWarning: `np.long` is a deprecated alias for `np.compat.long`. To silence this warning, use `np.compat.long` by itself. In the likely event your code does not need to work on Python 2 you can use the builtin `int` for which `np.compat.long` is itself an alias. Doing this will not modify any behaviour and is safe. When replacing `np.long`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  long_ = _make_signed(np.long)
/usr/local/lib/python3.7/dist-packages/scipy/fft/__init__.py:97: DeprecationWarning: The module numpy.dual is deprecated.  Instead of using dual, use the functions directly from numpy or scipy.
  from numpy.dual import register_func


## Passive

In [ ]:
passive_results = []

for seed in range(5):
    # create envs
    env = DummyVectorEnv([lambda: gym.make('Pendulum-v1')])
    env.seed(seed=0)

    # create the agent
    agent = sac_simple(
        'Pendulum-v1',
        train_collector=None,
        test_collector=None,
        observation_space=env.observation_space[0],
        action_space=env.action_space[0],
        init_passive_interface=True,
        step_per_epoch=1000,
        seed=seed
    )

    # prefill the buffer
    done = None
    obs = env.reset()

    for istep in range(128):
        act = [space.sample() for space in env.action_space]
        obs_next, rew, done, info = env.step(act)
        
        transition = Batch(obs=obs, obs_next=obs_next, rew=rew, done=done, info=info)
        transition.update(**Batch(act=act))

        agent.observe_transition(transition, training_enabled=False)

        obs = obs_next
        done_envs = np.where(done)[0]

        if len(done_envs):
            env.reset(done_envs)

    # run training
    state = None
    done = None
    obs = env.reset()

    for istep in range(10000):
        act_batch, state = agent.act(obs, state=state, done=done)
        obs_next, rew, done, info = env.step(act_batch.act)

        transition = Batch(obs=obs, obs_next=obs_next, rew=rew, done=done, info=info)
        transition.update(**act_batch, **state)
        agent.observe_transition(transition)

        obs = obs_next
        done_envs = np.where(done)[0]

        if len(done_envs):
            env.reset(done_envs)

    # run testing
    state = None
    done = None
    obs = env.reset()

    passive_collector = PassiveCollector(agent.policy)
    passive_collector.make_collect(n_episode=5)

    while True:
        act_batch, state = agent.act(obs, state=state, done=done, mode='eval')
        obs_next, rew, done, info = env.step(act_batch.act)

        transition = Batch(obs=obs, obs_next=obs_next, rew=rew, done=done, info=info)
        transition.update(**act_batch, **state)
        result = passive_collector.observe_transition(transition)

        obs = obs_next
        done_envs = np.where(done)[0]

        if len(done_envs):
            env.reset(done_envs)

        if result is not None:
            break

    print(f"rewards: {result['rew']:.6f} ± {result['rew_std']:.6f}")
    print(f"num episodes: {result['n/ep']}")

    passive_results.append(result)

C:\Users\Michal\anaconda3\lib\site-packages\gym\core.py:172: DeprecationWarning: WARN: Function `env.seed(seed)` is marked as deprecated and will be removed in the future. Please use `env.reset(seed=seed) instead.
  deprecation(
Epoch #1: 1001it [01:00, 16.49it/s, alpha=0.754, env_step=1000, len=200, loss/actor=25.749, loss/alpha=-0.642, loss/critic1=2.377, loss/critic2=2.363, n/ep=1, n/st=1, rew=-1744.86]                          
Epoch #2: 1001it [00:47, 21.16it/s, alpha=0.564, env_step=2000, len=200, loss/actor=57.189, loss/alpha=-1.200, loss/critic1=3.508, loss/critic2=3.589, n/ep=1, n/st=1, rew=-1519.57]                          
Epoch #3: 1001it [00:49, 20.04it/s, alpha=0.433, env_step=3000, len=200, loss/actor=77.267, loss/alpha=-1.342, loss/critic1=4.793, loss/critic2=5.405, n/ep=1, n/st=1, rew=-961.13]                          
Epoch #4: 1001it [00:49, 20.28it/s, alpha=0.342, env_step=4000, len=200, loss/actor=94.405, loss/alpha=-1.414, loss/critic1=6.861, loss/critic2=7.632, 

rewards: -149.877210 ± 122.860291
num episodes: 5


Epoch #1: 1001it [00:36, 27.43it/s, alpha=0.754, env_step=1000, len=200, loss/actor=25.714, loss/alpha=-0.642, loss/critic1=2.624, loss/critic2=2.688, n/ep=1, n/st=1, rew=-1651.46]                          
Epoch #2: 1001it [00:38, 25.96it/s, alpha=0.566, env_step=2000, len=200, loss/actor=55.829, loss/alpha=-1.144, loss/critic1=2.795, loss/critic2=2.758, n/ep=1, n/st=1, rew=-1494.87]                          
Epoch #3: 1001it [00:42, 23.28it/s, alpha=0.437, env_step=3000, len=200, loss/actor=76.741, loss/alpha=-1.262, loss/critic1=7.483, loss/critic2=7.941, n/ep=1, n/st=1, rew=-1136.20]                          
Epoch #4: 1001it [00:42, 23.63it/s, alpha=0.344, env_step=4000, len=200, loss/actor=93.093, loss/alpha=-1.397, loss/critic1=10.071, loss/critic2=10.321, n/ep=1, n/st=1, rew=-1304.23]                          
Epoch #5: 1001it [00:38, 25.68it/s, alpha=0.262, env_step=5000, len=200, loss/actor=111.854, loss/alpha=-1.848, loss/critic1=11.191, loss/critic2=11.370, n/ep=1, n/st=1, 

rewards: -169.844730 ± 158.844920
num episodes: 5


Epoch #1: 1001it [00:30, 33.24it/s, alpha=0.754, env_step=1000, len=200, loss/actor=22.104, loss/alpha=-0.638, loss/critic1=1.800, loss/critic2=1.678, n/ep=1, n/st=1, rew=-1626.38]                          
Epoch #2: 1001it [00:30, 32.33it/s, alpha=0.567, env_step=2000, len=200, loss/actor=51.109, loss/alpha=-1.125, loss/critic1=5.412, loss/critic2=5.383, n/ep=1, n/st=1, rew=-1431.40]                          
Epoch #3: 1001it [00:30, 33.17it/s, alpha=0.438, env_step=3000, len=200, loss/actor=73.159, loss/alpha=-1.280, loss/critic1=6.922, loss/critic2=7.138, n/ep=1, n/st=1, rew=-1068.32]                          
Epoch #4: 1001it [00:31, 31.65it/s, alpha=0.343, env_step=4000, len=200, loss/actor=89.217, loss/alpha=-1.385, loss/critic1=10.631, loss/critic2=9.646, n/ep=1, n/st=1, rew=-1271.89]                          
Epoch #5: 1001it [00:33, 29.91it/s, alpha=0.265, env_step=5000, len=200, loss/actor=107.644, loss/alpha=-1.672, loss/critic1=8.217, loss/critic2=7.198, n/ep=1, n/st=1, rew

rewards: -128.944983 ± 81.533843
num episodes: 5


Epoch #1: 1001it [00:29, 34.05it/s, alpha=0.753, env_step=1000, len=200, loss/actor=23.886, loss/alpha=-0.646, loss/critic1=3.216, loss/critic2=3.108, n/ep=1, n/st=1, rew=-1550.40]                          
Epoch #2: 1001it [00:30, 33.09it/s, alpha=0.565, env_step=2000, len=200, loss/actor=52.775, loss/alpha=-1.160, loss/critic1=3.036, loss/critic2=2.856, n/ep=1, n/st=1, rew=-1580.14]                          
Epoch #3: 1001it [00:30, 32.62it/s, alpha=0.437, env_step=3000, len=200, loss/actor=73.074, loss/alpha=-1.239, loss/critic1=4.916, loss/critic2=3.863, n/ep=1, n/st=1, rew=-1071.51]                          
Epoch #4: 1001it [00:30, 32.62it/s, alpha=0.347, env_step=4000, len=200, loss/actor=90.622, loss/alpha=-1.308, loss/critic1=5.000, loss/critic2=4.192, n/ep=1, n/st=1, rew=-1212.62]                          
Epoch #5: 1001it [00:30, 33.06it/s, alpha=0.268, env_step=5000, len=200, loss/actor=109.310, loss/alpha=-1.618, loss/critic1=6.780, loss/critic2=6.970, n/ep=1, n/st=1, rew=

rewards: -128.728912 ± 83.169117
num episodes: 5


Epoch #1: 1001it [00:30, 33.30it/s, alpha=0.753, env_step=1000, len=200, loss/actor=26.332, loss/alpha=-0.652, loss/critic1=2.924, loss/critic2=2.493, n/ep=1, n/st=1, rew=-1582.53]                          
Epoch #2: 1001it [00:30, 32.93it/s, alpha=0.566, env_step=2000, len=200, loss/actor=54.713, loss/alpha=-1.110, loss/critic1=4.621, loss/critic2=4.214, n/ep=1, n/st=1, rew=-1363.62]                          
Epoch #3: 1001it [00:31, 32.08it/s, alpha=0.441, env_step=3000, len=200, loss/actor=75.929, loss/alpha=-1.169, loss/critic1=8.214, loss/critic2=6.311, n/ep=1, n/st=1, rew=-770.29]                          
Epoch #4: 1001it [00:29, 33.38it/s, alpha=0.352, env_step=4000, len=200, loss/actor=93.368, loss/alpha=-1.239, loss/critic1=14.915, loss/critic2=6.919, n/ep=1, n/st=1, rew=-958.08]                           
Epoch #5: 1001it [00:30, 32.77it/s, alpha=0.270, env_step=5000, len=200, loss/actor=113.236, loss/alpha=-1.706, loss/critic1=8.532, loss/critic2=5.614, n/ep=1, n/st=1, rew=

rewards: -154.403355 ± 127.904609
num episodes: 5


In [ ]:
for r in passive_results:
    print(r['rew'], r['rew_std'])

-149.87721037442364 122.86029067291372
-169.84472997588384 158.8449199714876
-128.94498273113908 81.53384332857709
-128.72891152802816 83.16911748465036
-154.40335456581437 127.90460857226506


## Active

In [4]:
active_results = []

for seed in range(5):
    agent = sac_simple(
        'Pendulum-v1',
        step_per_epoch=1000,
        max_epoch=10,
        seed=seed
    )

    agent.train()

    result = agent.test()
    active_results.append(result)

    print(f"rewards: {result['rew']:.6f} ± {result['rew_std']:.6f}")
    print(f"num episodes: {result['n/ep']}")

/usr/local/lib/python3.7/dist-packages/gym/core.py:173: DeprecationWarning: WARN: Function `env.seed(seed)` is marked as deprecated and will be removed in the future. Please use `env.reset(seed=seed) instead.
  "Function `env.seed(seed)` is marked as deprecated and will be removed in the future. "
/usr/local/lib/python3.7/dist-packages/scipy/sparse/sputils.py:17: DeprecationWarning: `np.typeDict` is a deprecated alias for `np.sctypeDict`.
  supported_dtypes = [np.typeDict[x] for x in supported_dtypes]
/usr/local/lib/python3.7/dist-packages/numba/core/ir_utils.py:1525: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  if (hasattr(numpy, value)
/usr/local/lib/python3.7/dis

Epoch #1: test_reward: -1556.133689 ± 0.000000, best_reward: -1556.133689 ± 0.000000 in #1


Epoch #2: 1001it [00:19, 51.98it/s, alpha=0.566, env_step=2000, len=200, loss/actor=55.772, loss/alpha=-1.120, loss/critic1=0.308, loss/critic2=0.409, n/ep=0, n/st=1, rew=-1242.07]                          


Epoch #2: test_reward: -1071.445563 ± 0.000000, best_reward: -1071.445563 ± 0.000000 in #2


Epoch #3: 1001it [00:19, 50.25it/s, alpha=0.443, env_step=3000, len=200, loss/actor=74.183, loss/alpha=-1.130, loss/critic1=1.414, loss/critic2=1.992, n/ep=0, n/st=1, rew=-1059.42]                          


...

Epoch #8: test_reward: -132.098661 ± 0.000000, best_reward: -0.498322 ± 0.000000 in #6


Epoch #9: 1001it [00:21, 46.29it/s, alpha=0.118, env_step=9000, len=200, loss/actor=132.811, loss/alpha=-0.646, loss/critic1=7.854, loss/critic2=2.919, n/ep=0, n/st=1, rew=-239.22]


Epoch #9: test_reward: -126.290021 ± 0.000000, best_reward: -0.498322 ± 0.000000 in #6


Epoch #10: 1001it [00:20, 49.57it/s, alpha=0.100, env_step=10000, len=200, loss/actor=129.366, loss/alpha=-0.477, loss/critic1=9.226, loss/critic2=3.391, n/ep=0, n/st=1, rew=-236.82]


Epoch #10: test_reward: -242.565872 ± 0.000000, best_reward: -0.498322 ± 0.000000 in #6
rewards: -168.133855 ± 94.990764
num episodes: 5


In [5]:
for r in active_results:
    print(r['rew'], r['rew_std'])

-99.46480910152941 91.43535666664974
-94.6252016092109 89.35670481240048
-119.4260175561084 106.38124095227387
-165.91293719587853 92.18332791802898
-168.1338549757413 94.99076444554642
